# Prompt Templates

Um prompt para um modelo de linguagem é um conjunto de instruções ou entradas fornecidas por um usuário para guiar a resposta do modelo, ajudando-o a entender o contexto e gerar uma saída baseada em linguagem relevante e coerente, como responder a perguntas, completar frases ou participar de uma conversa.

In [1]:
from langchain_openai.llms import OpenAI

llm = OpenAI()

In [2]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template('''
Responda a seguinte pergunta do usuário:
{pergunta}                                   
''')
prompt_template

PromptTemplate(input_variables=['pergunta'], input_types={}, partial_variables={}, template='\nResponda a seguinte pergunta do usuário:\n{pergunta}                                   \n')

In [3]:
print(prompt_template.format(pergunta='O que é um buraco negro?'))


Responda a seguinte pergunta do usuário:
O que é um buraco negro?                                   



In [4]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template('''
Responda a seguinte pergunta do usuário em até {n_palvras} palavras:
{pergunta}                                   
''')
prompt_template.format(n_palvras=10, pergunta='O que é um buraco negro?')

'\nResponda a seguinte pergunta do usuário em até 10 palavras:\nO que é um buraco negro?                                   \n'

In [5]:
prompt_template = PromptTemplate.from_template('''
Responda a seguinte pergunta do usuário em até {n_palvras} palavras:
{pergunta}                                   
''', partial_variables={'n_palvras': 10})
prompt_template.format(pergunta='O que é um buraco negro?')

'\nResponda a seguinte pergunta do usuário em até 10 palavras:\nO que é um buraco negro?                                   \n'

In [6]:
prompt_template.format(n_palvras=5, pergunta='O que é um buraco negro?')

'\nResponda a seguinte pergunta do usuário em até 5 palavras:\nO que é um buraco negro?                                   \n'

## Composing prompts | Unindo múltiplos prompts

In [7]:
from langchain.prompts import PromptTemplate

template_word_count = PromptTemplate.from_template('''
Responda a pergunta em até {n_palavras} palavras.''')

template_lingua = PromptTemplate.from_template('''
Retorn a resposta na {lingua}.''')

template_final = (
    template_word_count
    + template_lingua
    + '\nResponda a pergunta seguinte seguindo as instruções: {pergunta}'
)

template_final.template

'\nResponda a pergunta em até {n_palavras} palavras.\nRetorn a resposta na {lingua}.\nResponda a pergunta seguinte seguindo as instruções: {pergunta}'

In [8]:
prompt = template_final.format(n_palavras=10, lingua='inglês', pergunta='O que é uma estrela?')
llm.invoke(prompt)

'\n\nA star is a luminous sphere of plasma held together by gravity.'

In [9]:
prompt = template_final.format(n_palavras=10, lingua='inglês', pergunta='O que é uma estrela?')
print(prompt)


Responda a pergunta em até 10 palavras.
Retorn a resposta na inglês.
Responda a pergunta seguinte seguindo as instruções: O que é uma estrela?


## Templates para Chat

In [10]:
from langchain.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_template('Essa é a minha dúvida: {duvida}')
chat_template.format_messages(duvida='Quem sou eu?')

[HumanMessage(content='Essa é a minha dúvida: Quem sou eu?', additional_kwargs={}, response_metadata={})]

In [11]:
from langchain.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        ('system', 'Você é um assistente engraçado e se chama {nome_assistente}'),
        ('human', 'Olá, como vai?'),
        ('ai', 'Melhor agora! como posso ajudá-lo?'),
        ('human', '{pergunta}')
    ]
)

chat_template

ChatPromptTemplate(input_variables=['nome_assistente', 'pergunta'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['nome_assistente'], input_types={}, partial_variables={}, template='Você é um assistente engraçado e se chama {nome_assistente}'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Olá, como vai?'), additional_kwargs={}), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Melhor agora! como posso ajudá-lo?'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['pergunta'], input_types={}, partial_variables={}, template='{pergunta}'), additional_kwargs={})])

In [12]:
chat_template.format_messages(nome_assistente='Asimo', pergunta='Qual o seu nome?')

[SystemMessage(content='Você é um assistente engraçado e se chama Asimo', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Olá, como vai?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Melhor agora! como posso ajudá-lo?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Qual o seu nome?', additional_kwargs={}, response_metadata={})]

In [13]:
from langchain_openai.chat_models import ChatOpenAI

chat = ChatOpenAI()

chat.invoke(chat_template.format_messages(nome_assistente='Asimo', pergunta='Qual o seu nome?'))

AIMessage(content='Meu nome é Asimo, o assistente engraçado! Como posso animar o seu dia?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 56, 'total_tokens': 81, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-6f9bf123-9be9-4526-8834-d77f20ba3486-0', usage_metadata={'input_tokens': 56, 'output_tokens': 25, 'total_tokens': 81, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

## Templates de Few-shot prompting para llm

In [14]:
exemplos = [
    {"pergunta": "Quem viveu mais tempo, Muhammad Ali ou Alan Turing?", 
     "resposta": 
     """São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: Quantos anos Muhammad Ali tinha quando morreu? 
Resposta intermediária: Muhammad Ali tinha 74 anos quando morreu. 
Pergunta de acompanhamento: Quantos anos Alan Turing tinha quando morreu? 
Resposta intermediária: Alan Turing tinha 41 anos quando morreu. 
Então a resposta final é: Muhammad Ali 
""", 
    }, 
    {"pergunta": "Quando nasceu o fundador do craigslist?", 
     "resposta": 
"""São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: Quem foi o fundador do craigslist? 
Resposta intermediária: O craigslist foi fundado por Craig Newmark. 
Pergunta de acompanhamento: Quando nasceu Craig Newmark? 
Resposta intermediária: Craig Newmark nasceu em 6 de dezembro de 1952. 
Então a resposta final é: 6 de dezembro de 1952 
""", 
    }, 
    {"pergunta": "Quem foi o avô materno de George Washington?",
     "resposta": 
"""São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: Quem foi a mãe de George Washington? 
Resposta intermediária: A mãe de George Washington foi Mary Ball Washington. 
Pergunta de acompanhamento: Quem foi o pai de Mary Ball Washington? 
Resposta intermediária: O pai de Mary Ball Washington foi Joseph Ball. 
Então a resposta final é: Joseph Ball 
""", 
    },
    {"pergunta": "Os diretores de Jaws e Casino Royale são do mesmo país?", 
     "resposta": 
"""São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: Quem é o diretor de Jaws? 
Resposta Intermediária: O diretor de Jaws é Steven Spielberg. 
Pergunta de acompanhamento: De onde é Steven Spielberg? 
Resposta Intermediária: Estados Unidos. 
Pergunta de acompanhamento: Quem é o diretor de Casino Royale? 
Resposta Intermediária: O diretor de Casino Royale é Martin Campbell. 
Pergunta de acompanhamento: De onde é Martin Campbell? 
Resposta Intermediária: Nova Zelândia. 
Então a resposta final é: Não 
""",
    },
]

In [15]:
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate

example_prompt = PromptTemplate(
    input_variables=['pergunta', 'resposta'],
    template='Pergunta {pergunta}\n{resposta}'
)

example_prompt.format(**exemplos[0])


'Pergunta Quem viveu mais tempo, Muhammad Ali ou Alan Turing?\nSão necessárias perguntas de acompanhamento aqui: Sim. \nPergunta de acompanhamento: Quantos anos Muhammad Ali tinha quando morreu? \nResposta intermediária: Muhammad Ali tinha 74 anos quando morreu. \nPergunta de acompanhamento: Quantos anos Alan Turing tinha quando morreu? \nResposta intermediária: Alan Turing tinha 41 anos quando morreu. \nEntão a resposta final é: Muhammad Ali \n'

In [16]:
prompt = FewShotPromptTemplate(
    examples=exemplos,
    example_prompt=example_prompt,
    suffix='Pergunta: {input}',
    input_variables=['input']
)
prompt

FewShotPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, examples=[{'pergunta': 'Quem viveu mais tempo, Muhammad Ali ou Alan Turing?', 'resposta': 'São necessárias perguntas de acompanhamento aqui: Sim. \nPergunta de acompanhamento: Quantos anos Muhammad Ali tinha quando morreu? \nResposta intermediária: Muhammad Ali tinha 74 anos quando morreu. \nPergunta de acompanhamento: Quantos anos Alan Turing tinha quando morreu? \nResposta intermediária: Alan Turing tinha 41 anos quando morreu. \nEntão a resposta final é: Muhammad Ali \n'}, {'pergunta': 'Quando nasceu o fundador do craigslist?', 'resposta': 'São necessárias perguntas de acompanhamento aqui: Sim. \nPergunta de acompanhamento: Quem foi o fundador do craigslist? \nResposta intermediária: O craigslist foi fundado por Craig Newmark. \nPergunta de acompanhamento: Quando nasceu Craig Newmark? \nResposta intermediária: Craig Newmark nasceu em 6 de dezembro de 1952. \nEntão a resposta final é: 6 de dezembro

In [17]:
print(prompt.format(input='Quem fez mais gols, Romário ou Pelé?'))

Pergunta Quem viveu mais tempo, Muhammad Ali ou Alan Turing?
São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: Quantos anos Muhammad Ali tinha quando morreu? 
Resposta intermediária: Muhammad Ali tinha 74 anos quando morreu. 
Pergunta de acompanhamento: Quantos anos Alan Turing tinha quando morreu? 
Resposta intermediária: Alan Turing tinha 41 anos quando morreu. 
Então a resposta final é: Muhammad Ali 


Pergunta Quando nasceu o fundador do craigslist?
São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: Quem foi o fundador do craigslist? 
Resposta intermediária: O craigslist foi fundado por Craig Newmark. 
Pergunta de acompanhamento: Quando nasceu Craig Newmark? 
Resposta intermediária: Craig Newmark nasceu em 6 de dezembro de 1952. 
Então a resposta final é: 6 de dezembro de 1952 


Pergunta Quem foi o avô materno de George Washington?
São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: 

In [18]:
llm.invoke(prompt.format(input='Quem fez mais gols, Romário ou Pelé?'))

'\nSão necessárias perguntas de acompanhamento aqui: Não. \nResposta final: Pelé fez mais gols, com um total de 1.283 em sua carreira, enquanto Romário fez 772 gols.'

## Templates de Few-shot prompting para chat

In [19]:
from langchain_openai.chat_models import ChatOpenAI

chat = ChatOpenAI()



In [20]:
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.prompts import ChatPromptTemplate

example_prompt = ChatPromptTemplate.from_messages(
    [('human', '{pergunta}'),
     ('ai', '{resposta}')]
)

print(example_prompt.format_messages(**exemplos[0]))

[HumanMessage(content='Quem viveu mais tempo, Muhammad Ali ou Alan Turing?', additional_kwargs={}, response_metadata={}), AIMessage(content='São necessárias perguntas de acompanhamento aqui: Sim. \nPergunta de acompanhamento: Quantos anos Muhammad Ali tinha quando morreu? \nResposta intermediária: Muhammad Ali tinha 74 anos quando morreu. \nPergunta de acompanhamento: Quantos anos Alan Turing tinha quando morreu? \nResposta intermediária: Alan Turing tinha 41 anos quando morreu. \nEntão a resposta final é: Muhammad Ali \n', additional_kwargs={}, response_metadata={})]


In [21]:
few_shot_template = FewShotChatMessagePromptTemplate(
    examples=exemplos,
    example_prompt=example_prompt
)

prompt_template = ChatPromptTemplate.from_messages(
    [
    few_shot_template,
    ('human', '{input}')
    ]
)

prompt = prompt_template.format_messages(input='Quem fez mais gols, Romário ou Pelé?')
prompt

[HumanMessage(content='Quem viveu mais tempo, Muhammad Ali ou Alan Turing?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='São necessárias perguntas de acompanhamento aqui: Sim. \nPergunta de acompanhamento: Quantos anos Muhammad Ali tinha quando morreu? \nResposta intermediária: Muhammad Ali tinha 74 anos quando morreu. \nPergunta de acompanhamento: Quantos anos Alan Turing tinha quando morreu? \nResposta intermediária: Alan Turing tinha 41 anos quando morreu. \nEntão a resposta final é: Muhammad Ali \n', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Quando nasceu o fundador do craigslist?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='São necessárias perguntas de acompanhamento aqui: Sim. \nPergunta de acompanhamento: Quem foi o fundador do craigslist? \nResposta intermediária: O craigslist foi fundado por Craig Newmark. \nPergunta de acompanhamento: Quando nasceu Craig Newmark? \nResposta intermediária: Craig Newmark nasce

In [22]:
chat.invoke(prompt)

AIMessage(content='Pelé é amplamente considerado um dos maiores jogadores de futebol de todos os tempos, com um incrível recorde de gols marcados ao longo de sua carreira. Romário, por outro lado, também foi um jogador excepcional e conhecido por sua habilidade de marcar gols. No entanto, Pelé marcou mais gols ao longo de sua carreira do que Romário. Pelé é conhecido por ter marcado mais de 1000 gols em jogos oficiais, enquanto Romário marcou cerca de 740 gols em sua carreira. Portanto, Pelé fez mais gols do que Romário.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 147, 'prompt_tokens': 538, 'total_tokens': 685, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-a7a4f0f3